In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-128915")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-128915
Azure region: southcentralus
Subscription id: a1ebbd25-46bc-446b-9547-5acab9b0125a
Resource group: aml-quickstarts-128915


In [10]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# TODO: Create compute cluster

cpu_cluster_name = 'cpucluster'

# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)


### YOUR CODE HERE ###

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [32]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({"--C" : uniform(0.1,1),
                            "--max_iter": choice(50,100,150,200)})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1,evaluation_interval = 1,delay_evaluation=5)
### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory = "./training",entry_script="train.py",compute_target=cpu_cluster_name)
### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config =  HyperDriveConfig(
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    max_total_runs=100,
    max_concurrent_runs=4,
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    policy=policy,
    estimator=est
)

### YOUR CODE HERE ###

In [33]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

experiment=Experiment(ws,ws.name)
hiperdrive_run = experiment.submit(hyperdrive_config)

RunDetails(hiperdrive_run).show()
### YOUR CODE HERE ###

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [27]:
import joblib
# Get your best run and save the model from that run.
best_run = hiperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run :' , best_run.id)
print('\nAccuracy:' , best_run_metrics['accuracy'])
print('\nBest Run :' ,  parameter_values[3])
print('\nBest Run :' , parameter_values[5])
print('\nBest Run :' , parameter_values[7])




### YOUR CODE HERE ###

AttributeError: 'NoneType' object has no attribute 'get_metrics'

In [29]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
from azureml.core import Workspace, Dataset

subscription_id = 'a1ebbd25-46bc-446b-9547-5acab9b0125a'
resource_group = 'aml-quickstarts-128915'
workspace_name = 'quick-starts-ws-128915'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='Bank-marketing')
ds = dataset.to_pandas_dataframe()

### YOUR CODE HERE ###

In [30]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

NameError: name 'clean_data' is not defined

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###